# 02 Transform - Create Silver & Gold Layers

This notebook transforms Bronze tables into:
- **Silver Layer**: Cleaned data
- **Gold Layer**: Business-ready analytics tables

**Prerequisites:**
- `01_Extract` notebook completed
- Bronze tables exist: bronze_products, bronze_sales, bronze_customers

## Load Bronze Tables

In [ ]:
from pyspark.sql import functions as F

# Load bronze tables (use direct paths)
products = spark.read.format("delta").load("Tables/bronze_products")
sales = spark.read.format("delta").load("Tables/bronze_sales")
customers = spark.read.format("delta").load("Tables/bronze_customers")

print(f"✓ Loaded {products.count()} products")
print(f"✓ Loaded {sales.count()} sales")
print(f"✓ Loaded {customers.count()} customers")

## Gold Layer: Sales by Product

In [ ]:
# Join sales with products
sales_by_product = sales.join(products, "ProductKey", "left") \
    .groupBy("ProductKey", "ProductName") \
    .agg(
        F.sum("SalesAmount").alias("TotalSales"),
        F.sum("OrderQuantity").alias("TotalQuantity"),
        F.count("SalesOrderNumber").alias("OrderCount")
    ) \
    .orderBy(F.desc("TotalSales"))

# Save as Gold table
sales_by_product.write.mode("overwrite").format("delta").save("Tables/gold_sales_by_product")

print(f"✓ Created Sales by Product: {sales_by_product.count()} products")
sales_by_product.show(10)

## Gold Layer: Sales Trend

In [ ]:
# Sales trend by month
sales_trend = sales \
    .withColumn("YearMonth", F.date_format("OrderDate", "yyyy-MM")) \
    .groupBy("YearMonth") \
    .agg(
        F.sum("SalesAmount").alias("TotalSales"),
        F.count("SalesOrderNumber").alias("OrderCount")
    ) \
    .orderBy("YearMonth")

# Save as Gold table
sales_trend.write.mode("overwrite").format("delta").save("Tables/gold_sales_trend")

print(f"✓ Created Sales Trend: {sales_trend.count()} months")
sales_trend.show()

## Gold Layer: Top Customers

In [ ]:
# Top customers
top_customers = sales.join(customers, "CustomerKey", "left") \
    .withColumn("CustomerName",
                F.concat_ws(" ", F.col("FirstName"), F.col("LastName"))) \
    .groupBy("CustomerKey", "CustomerName", "EmailAddress") \
    .agg(
        F.sum("SalesAmount").alias("TotalPurchases"),
        F.count("SalesOrderNumber").alias("OrderCount")
    ) \
    .orderBy(F.desc("TotalPurchases")) \
    .limit(100)

# Save as Gold table
top_customers.write.mode("overwrite").format("delta").save("Tables/gold_top_customers")

print(f"✓ Created Top 100 Customers")
top_customers.show(10)

## Summary

✅ Transform complete! Three gold tables created:
- `gold_sales_by_product`
- `gold_sales_trend`
- `gold_top_customers`

**Next step:** Create SQL Warehouse and load these tables for querying.